In [56]:
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import KFold, cross_val_score, train_test_split,StratifiedKFold # Model evaluation
from sklearn.preprocessing import LabelEncoder, RobustScaler, OneHotEncoder, StandardScaler # Preprocessing
from sklearn.linear_model import Lasso, Ridge, ElasticNet,  LassoLarsIC, RANSACRegressor, SGDRegressor, HuberRegressor, BayesianRidge # Linear models
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor  # Ensemble methods
from xgboost import XGBRegressor, plot_importance # XGBoost
from sklearn.svm import SVR, SVC, LinearSVC  # Support Vector Regression
from sklearn.tree import DecisionTreeRegressor # Decision Tree Regression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline # Streaming pipelines
from sklearn.decomposition import KernelPCA, PCA # Dimensionality reduction
from sklearn.feature_selection import SelectFromModel # Dimensionality reduction
from sklearn.model_selection import learning_curve, validation_curve, GridSearchCV # Model evaluation
from sklearn.base import clone, BaseEstimator, TransformerMixin, RegressorMixin # Clone estimator
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import explained_variance_score, f1_score, roc_auc_score, median_absolute_error, r2_score, mean_squared_error #To evaluate our model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, fbeta_score #To evaluate our model
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
import optuna

In [57]:
import pandas as pd
import numpy as np
import re
import numpy as np

In [58]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
testId = test.Property_Id
train.Price_Category = train.Price_Category.replace("Cheap",0).replace("Affordable",1).replace("Semi-Premium",2).replace("Premium",3)
y = train.Price_Category
train = train.drop(["Property_Id", "Price_Category"],axis=1)
test = test.drop(["Property_Id"],axis=1)
features = pd.concat([train,test])
train.head(1)

,Title,Label,Type,Location,Baths,Area,Purpose,Bedroom(s),Description,Main Features,Rooms,Business and Communication,Healthcare Recreational,Nearby Locations and Other Facilities,Other Facilities,Popular,Source,Time Stamp
0,Brand New 1 Kanal House For Sale In Bahria Tow...,"Bahria Town - Gulbahar Block, Bahria Town - Se...",House,"Bahria Town, Lahore, Punjab",6,1 Kanal,For Sale,5,Brand New 1 Kanal House For Sale In Bahria Tow...,"Built in year: 2018 , Parking Spaces: 4 , Dou...","Bedrooms: 5 , Bathrooms: 7 , Servant Quarters...","Broadband Internet Access , Satellite or Cabl...","Lawn or Garden , Swimming Pool","Nearby Schools , Nearby Hospitals , Nearby Sh...","Maintenance Staff , Security Staff , Faciliti...",super hot,Lahore-1-10,3/31/2019 11:01


In [59]:
def findWcolon(a,x, length):
    val = re.findall("%s: \d+" % a,x)
    val = val[0][-length:] if len(val)>0 else (np.nan if len(x)==1 else 0)
    return val
def find(a,x):
    return 1 if a in x else (np.nan if len(x)==1 else 0)

def rooms(x):
    living = find("Study Room",x) + find("Drawing Room",x) + find("Dining Room",x) + find("Lounge or Sitting Room", x)
    athletic = find("Gym",x) + find("Steam Room",x) + find("Laundry Room",x) + find("Powder Room", x)
    #findWcolon("Bathrooms",x,1),
    return [findWcolon("Servant Quarters",x,1), findWcolon("Kitchens",x,1), findWcolon("Store Rooms",x,1), living, athletic, find("Other Rooms", x), find("Steam Room",x), find("Laundry Room",x), find("Powder Room", x), find("Study Room",x), find("Drawing Room",x), find("Dining Room",x), find("Lounge or Sitting Room", x)]

def main_features(x):
    return [findWcolon("Built in year",x,4), findWcolon("Parking Spaces",x,1), findWcolon("Floors",x,1), find("Double Glazed Windows", x), find("Flooring", x), find("Electricity Backup", x), find("Waste Disposal", x),find("Central Air Conditioning", x), find("Flooring", x)]

def business(x):
    return [find("Broadband Internet Access", x), find("Satellite or Cable TV Ready", x), find("Intercom", x), find("Other Business and Communication Facilities", x)]

def health(x):
    return [find("Lawn or Garden", x), find("Sauna", x) + find("Jacuzzi", x), find("Swimming Pool", x), find("Other Healthcare and Recreation Facilities", x)]

def facilities(x):
    cols = ['Maintenance Staff','Security Staff','Facilities for Disabled','Other Facilities']
    return [find(y,x) for y in cols]

def nearby(x):
    val = len([y for y in x.split(" , ") if "Nearby" in y]) if x != " " else np.nan
    return val


def process(train):
    train["nearbyVal"] = [1 if len(x) > 1 else 0 for x in train["Nearby Locations and Other Facilities"]]
    train["bussinessVal"] = [1 if len(x) > 1 else 0 for x in train["Business and Communication"]]
    train["staffVal"] = [1 if len(x) > 1 else 0 for x in train["Other Facilities"]]
    train["healthVal"] = [1 if len(x) > 1 else 0 for x in train["Healthcare Recreational"]]
    train[["Date","Parking","Floors","Windows", "Flooring", "ElectricBackup", "WasteDisposal", "CentralAir", "CentralHeating"]] = list(train["Main Features"].apply(main_features))
    train[["Servants","Kitchens","Store Rooms", "living","healthRooms","OtherRooms", "Steam Room","Laundry Room","Powder Room","Study Room","Drawing Room","Dining Room","Lounge"]] = list(train.Rooms.apply(rooms))
    train[["internet","tv","intercom","otherBusiness"]] = list(train["Business and Communication"].apply(business))
    train[["town","city","province"]] = list(train.Location.apply(lambda x: x.split(",")))
    train[["lawn","hotTub","pool","otherHealth"]] = list(train["Healthcare Recreational"].apply(health))
    train.Area = ["".join(x.split(",")) for x in train.Area]
    train.Area = train.Area.apply(lambda x: float(x.split(" ")[0]) if x.split(" ")[1] == "Marla" else float(x.split(" ")[0]) *20)
    train[['MaintenanceStaff','SecurityStaff','FacilitiesDisabled','ExtraFacilities']] = list(train["Other Facilities"].apply(facilities))
    train["nearby2"] = train["Nearby Locations and Other Facilities"].apply(nearby)
    train.Popular = train.Popular.replace("hot",1).replace("super hot",2)
    train = pd.concat([train,pd.get_dummies(train.Location)],axis=1)
    train[["Source", "SourceNum"]] = [["-".join(x.split("-")[:-1]), x.split("-")[-1] ]for x in train.Source]
    train = pd.concat([train,pd.get_dummies(train.Source)],axis=1)
    train = train.replace("-",0)
    train = train.drop(["Title","Label","Date", "city", "province", "town","Type","Purpose", "Healthcare Recreational", "Other Facilities", "Rooms","Nearby Locations and Other Facilities","Main Features", "Source", "Time Stamp", "Description", "Location", "Business and Communication"],axis=1)
    train = train.fillna(0)
    train = train.astype('int')
    return train

In [60]:
features = process(features)
train = features[:len(y)]
test = features[len(y):]
train = train.values
test = test.values


In [69]:
#Set to False if you want to skip it

OPTUNA_OPTIMIZATION = True
N_SPLITS = 10 #Number of folds for validation
N_TRIALS = 50 #Number of trials to find best hyperparameters

In [70]:
def objective(trial, cv=StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)):
    
    
    param_lgb = {
        "random_state": trial.suggest_int("random_state", 1, 100),
        "objective": "multiclass",
        "verbosity": -1,
        "boosting_type": "gbdt",
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "max_depth": trial.suggest_int("max_depth", -1, 10),
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.4, 1.0),
        "subsample": trial.suggest_float("subsample", 0.4, 1.0),
        "subsample_freq": trial.suggest_int("subsample_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }
    
    model = LGBMClassifier(**param_lgb)
    
    val_f1 = []
    f1s = []
    
    for kfold, (train_idx, val_idx) in enumerate(cv.split(train, y)):
        
        model.fit(train[train_idx], y[train_idx])
        print('Fitted {}'.format(type(model).__name__))
        val_true = y[val_idx].values
        
        preds = model.predict(train[val_idx]).round(0)
        
        f1 = f1_score(val_true, preds, average='macro')
        
        print('Fold: {}\t F1: {}\n'.format(kfold, f1))
        f1s.append(f1)
    
    print('Average F1: {}'.format(np.average(f1)))
    return np.average(f1)

In [71]:
if OPTUNA_OPTIMIZATION:
    study = optuna.create_study(study_name = 'lgbm_parameter_opt', direction="maximize")
    study.optimize(objective, n_trials=N_TRIALS) 
    
    trial = study.best_trial
    
    print("  Value: {}".format(trial.value))
    
    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))
else:
    trial = {"reg_alpha": 0.362136938773081,
             "reg_lambda": 2.930297242488071,
             "max_depth": 10,
             "n_estimators": 306,
             "num_leaves": 71,
             "colsample_bytree": 0.7121396258381646,
             "subsample": 0.793959734582999,
             "subsample_freq": 2,
             "min_child_samples": 18}

[I 2021-03-22 20:22:41,728] A new study created in memory with name: lgbm_parameter_opt


Fitted LGBMClassifier
Fold: 0	 AUC: 0.6362556814544523

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7193396226415094

Fitted LGBMClassifier
Fold: 2	 AUC: 0.5992844106401076

Fitted LGBMClassifier
Fold: 3	 AUC: 0.7644927536231884

Fitted LGBMClassifier
Fold: 4	 AUC: 0.6156588509529687

Fitted LGBMClassifier
Fold: 5	 AUC: 0.5969061794105005

Fitted LGBMClassifier
Fold: 6	 AUC: 0.6333798572649478

Fitted LGBMClassifier
Fold: 7	 AUC: 0.5726163171690695

Fitted LGBMClassifier
Fold: 8	 AUC: 0.6968558409279205



[I 2021-03-22 20:24:14,666] Trial 0 finished with value: 0.6594596164588991 and parameters: {'random_state': 99, 'reg_alpha': 0.0006276843122715588, 'reg_lambda': 0.01638563491998784, 'max_depth': 0, 'n_estimators': 219, 'num_leaves': 94, 'colsample_bytree': 0.49348794880557595, 'subsample': 0.6783993544664527, 'subsample_freq': 2, 'min_child_samples': 93}. Best is trial 0 with value: 0.6594596164588991.


Fitted LGBMClassifier
Fold: 9	 AUC: 0.7598066505043249

Average AUC: 0.7598066505043249
Fitted LGBMClassifier
Fold: 0	 AUC: 0.594440983039114

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7082639437748179

Fitted LGBMClassifier
Fold: 2	 AUC: 0.5424297924297925

Fitted LGBMClassifier
Fold: 3	 AUC: 0.763172798550157

Fitted LGBMClassifier
Fold: 4	 AUC: 0.6115585807020861

Fitted LGBMClassifier
Fold: 5	 AUC: 0.5971804964545436

Fitted LGBMClassifier
Fold: 6	 AUC: 0.6438972626472627

Fitted LGBMClassifier
Fold: 7	 AUC: 0.5384292606625931

Fitted LGBMClassifier
Fold: 8	 AUC: 0.653624154925025



[I 2021-03-22 20:24:52,035] Trial 1 finished with value: 0.6404130047045438 and parameters: {'random_state': 39, 'reg_alpha': 0.09842432108849983, 'reg_lambda': 0.0002367443557973528, 'max_depth': 10, 'n_estimators': 197, 'num_leaves': 106, 'colsample_bytree': 0.8122906240578653, 'subsample': 0.41090230098818603, 'subsample_freq': 7, 'min_child_samples': 81}. Best is trial 0 with value: 0.6594596164588991.


...

Fitted LGBMClassifier
Fold: 9	 AUC: 0.7740959269128282

Average AUC: 0.7740959269128282
Fitted LGBMClassifier
Fold: 0	 AUC: 0.6287756573600494

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7627143432921566

Fitted LGBMClassifier
Fold: 2	 AUC: 0.6148127250635844

Fitted LGBMClassifier
Fold: 3	 AUC: 0.8085755739632868

Fitted LGBMClassifier
Fold: 4	 AUC: 0.5924251627573747

Fitted LGBMClassifier
Fold: 5	 AUC: 0.7767923438660176

Fitted LGBMClassifier
Fold: 6	 AUC: 0.6319822496990046

Fitted LGBMClassifier
Fold: 7	 AUC: 0.6093418875561097

Fitted LGBMClassifier
Fold: 8	 AUC: 0.7969632904093793



[I 2021-03-22 22:23:13,227] Trial 48 finished with value: 0.7012003973038625 and parameters: {'random_state': 1, 'reg_alpha': 5.4469656365239525e-06, 'reg_lambda': 8.756315837844564e-07, 'max_depth': 5, 'n_estimators': 500, 'num_leaves': 140, 'colsample_bytree': 0.6551945036576825, 'subsample': 0.7506842319301558, 'subsample_freq': 1, 'min_child_samples': 8}. Best is trial 22 with value: 0.739505989820379.


Fitted LGBMClassifier
Fold: 9	 AUC: 0.7896207390716625

Average AUC: 0.7896207390716625
Fitted LGBMClassifier
Fold: 0	 AUC: 0.6223065691867231

Fitted LGBMClassifier
Fold: 1	 AUC: 0.7389972899728997

Fitted LGBMClassifier
Fold: 2	 AUC: 0.5997860671146942

Fitted LGBMClassifier
Fold: 3	 AUC: 0.7843889065479974

Fitted LGBMClassifier
Fold: 4	 AUC: 0.6243727863046045

Fitted LGBMClassifier
Fold: 5	 AUC: 0.5871783681123416

Fitted LGBMClassifier
Fold: 6	 AUC: 0.6358668452881769

Fitted LGBMClassifier
Fold: 7	 AUC: 0.77326613229698

Fitted LGBMClassifier
Fold: 8	 AUC: 0.7869294229394123



[I 2021-03-22 22:23:30,733] Trial 49 finished with value: 0.6940933917818474 and parameters: {'random_state': 12, 'reg_alpha': 1.6637392517964533e-05, 'reg_lambda': 0.0004395001386254016, 'max_depth': 1, 'n_estimators': 338, 'num_leaves': 197, 'colsample_bytree': 0.6014733537928333, 'subsample': 0.8901392310749766, 'subsample_freq': 2, 'min_child_samples': 17}. Best is trial 22 with value: 0.739505989820379.


Fitted LGBMClassifier
Fold: 9	 AUC: 0.7878415300546449

Average AUC: 0.7878415300546449
  Value: 0.739505989820379
  Params: 
    random_state: 10
    reg_alpha: 0.00011427811031850062
    reg_lambda: 9.702768925062732e-07
    max_depth: 6
    n_estimators: 400
    num_leaves: 180
    colsample_bytree: 0.5515781349051045
    subsample: 0.8354989110263507
    subsample_freq: 3
    min_child_samples: 7


In [72]:
trial.params

{'random_state': 10,
 'reg_alpha': 0.00011427811031850062,
 'reg_lambda': 9.702768925062732e-07,
 'max_depth': 6,
 'n_estimators': 400,
 'num_leaves': 180,
 'colsample_bytree': 0.5515781349051045,
 'subsample': 0.8354989110263507,
 'subsample_freq': 3,
 'min_child_samples': 7}

In [77]:
if OPTUNA_OPTIMIZATION:
    final_model = LGBMRegressor(**trial.params)
else:
    final_model = LGBMRegressor(**trial)

In [85]:
test_preds = []

skf = StratifiedKFold(N_SPLITS, shuffle = True, random_state = 29)
f1s = []
for kfold, (train_idx, val_idx) in enumerate(skf.split(train, 
                                                      y)):
        
        final_model.fit(train[train_idx], y.loc[train_idx])
        print('Fitted {}'.format(type(final_model).__name__))
        val_true =y[val_idx]
        
        preds = final_model.predict(train[val_idx]).round(0)
        
        f1 = f1_score(val_true, preds, average='macro')
        f1s.append(f1)
        print('Fold: {}\t Validation F1: {}\n'.format(kfold, f1))
        
        test_preds.append(final_model.predict(test))
        
print("Best Parameters mean AUC: {}".format(np.mean(f1s)))

Fitted LGBMRegressor
Fold: 0	 Validation F1: 0.8131494938546558

Fitted LGBMRegressor
Fold: 1	 Validation F1: 0.7479204706837086

Fitted LGBMRegressor
Fold: 2	 Validation F1: 0.6265808711281114

Fitted LGBMRegressor
Fold: 3	 Validation F1: 0.7705089315807212

Fitted LGBMRegressor
Fold: 4	 Validation F1: 0.4989235873667148

Fitted LGBMRegressor
Fold: 5	 Validation F1: 0.8022844418077785

Fitted LGBMRegressor
Fold: 6	 Validation F1: 0.6369475197948611

Fitted LGBMRegressor
Fold: 7	 Validation F1: 0.5679348342357746

Fitted LGBMRegressor
Fold: 8	 Validation F1: 0.7490629021879022

Fitted LGBMRegressor
Fold: 9	 Validation F1: 0.7967970548698875

Best Parameters mean AUC: 0.7010110107510116


In [8]:
def run_model(name, model,train,y,test):
    X_train, X_test, y_train, y_test = train_test_split(train, y, test_size=0.33, random_state=42)
    model2 = model
    model2.fit(X_train,y_train)
    predictions = model.predict(X_test).round(0)
    print("%s F1 %s" % (name,f1_score(y_test, predictions, average='macro')))
    model.fit(train,y)
    preds = model.predict(test).round(0)
    return preds

In [9]:
models = [
    ("KNN",KNeighborsClassifier(n_neighbors=5)),
    ("LGBM",LGBMRegressor()),
    ("XGB",XGBRegressor())
]
for name,model in models:
    run_model(name, model, train,y,test)

KNN F1 0.5517214506116731
LGBM F1 0.7340929540637331
XGB F1 0.6937436378138908


In [18]:
name = models[1][0]
model = models[1][1]
preds = run_model(name, model, train,y,test)

LGBM F1 0.7340929540637331


In [86]:
test_predictions = np.mean(test_preds, axis = 0)


array([2.91450815, 1.98367779, 2.17311202, 2.83456717, 2.02183598,
       1.56481898, 2.11472169, 2.13107427, 2.01028159, 1.89915841,
       1.62435185, 1.99865341, 2.96229537, 1.97555445, 2.00116851,
       2.81122846, 2.20644026, 1.27551838, 2.01338936, 3.00540062,
       1.95015101, 2.97844978, 3.01703732, 2.1959208 , 2.0308977 ,
       1.96838308, 2.8791071 , 2.24351726, 2.00854347, 2.03458581,
       2.0607298 , 1.98514151, 2.77268006, 2.79939361, 2.5638217 ,
       1.2029357 , 2.31075907, 1.96818203, 1.93358184, 2.91594943,
       2.00900078, 2.00814001, 2.14112084, 2.11924374, 3.00034336,
       1.99061595, 1.9536701 , 1.98284323, 1.99022693, 2.87164442,
       1.97148265, 1.76836628, 2.53427971, 2.06376437, 2.99900846,
       2.05809566, 2.84339689, 2.97335635, 2.5840912 , 1.86359443,
       2.99355071, 1.88965314, 2.81705914, 1.15014612, 1.98762376,
       3.00037236, 2.83152305, 2.05950464, 1.90577609, 2.95831334,
       2.01463304, 2.05239447, 2.11686671, 2.38592448, 2.02052

In [87]:
preds = [["Cheap","Affordable","Semi-Premium","Premium"][int(x)] for x in test_predictions.round(0)]

In [88]:
sub = pd.DataFrame({"Property_Id":testId,"Price_Category":preds})

In [89]:
sub.to_csv("../output/sub_with_optuna.csv",index=False)

In [98]:
pre_sub = pd.read_csv("../output/sub_optuna1.csv")
pre_sub.Price_Category.value_counts()/len(pre_sub)

Semi-Premium    0.594495
Premium         0.365138
Affordable      0.040367
Name: Price_Category, dtype: float64

In [99]:
sub["Price_Category"].value_counts()/len(sub)

Semi-Premium    0.587156
Premium         0.363303
Affordable      0.049541
Name: Price_Category, dtype: float64

In [100]:
pd.concat([sub,pre_sub],axis=1)[sub.Price_Category != pre_sub.Price_Category]

,Property_Id,Price_Category,Property_Id,Price_Category
3,3000,Premium,3000,Semi-Premium
52,711,Premium,711,Semi-Premium
84,221,Affordable,221,Semi-Premium
150,867,Affordable,867,Semi-Premium
176,3612,Affordable,3612,Semi-Premium
178,523,Premium,523,Semi-Premium
213,940,Affordable,940,Semi-Premium
223,2285,Affordable,2285,Semi-Premium
243,2540,Affordable,2540,Semi-Premium
269,2576,Premium,2576,Semi-Premium
